In [12]:
import os
import json
import zipfile
import random
from PIL import Image
import io

# --- נתיבי מקור ---
base_path = r"C:\Users\linoy\OneDrive\שולחן העבודה\Samsung"
json_path = os.path.join(base_path, "annotations.json")
zip_path = os.path.join(base_path, "crops.zip")

# --- נתיב יעד ---
output_folder = "distorted_30x30_dataset" 

# --- הגדרות עיבוד (הגדלתי את ה-MAX כדי שלא יפסול הכל) ---
TARGET_SIZE = (30, 30)
MIN_ORIGINAL_SIZE = 30
MAX_ORIGINAL_SIZE = 5000  # הגדלתי ל-5000 ליתר ביטחון
SAMPLES_PER_ARTIFACT = 100

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

with open(json_path, 'r', encoding='utf-8') as f:
    annotations = json.load(f)

artifact_groups = {}
for entry in annotations:
    img_id = list(entry.keys())[0]
    artifact_type = entry[img_id]['artifact']
    if artifact_type not in artifact_groups:
        artifact_groups[artifact_type] = []
    artifact_groups[artifact_type].append(img_id)

print(f"סורק את קובץ הזיפ: {zip_path}...")

with zipfile.ZipFile(zip_path, 'r') as archive:
    # יוצר רשימה של כל הנתיבים בתוך הזיפ
    all_names = archive.namelist()
    
    for artifact_type, ids in artifact_groups.items():
        random.shuffle(ids)
        extracted_count = 0
        
        for img_id in ids:
            if extracted_count >= SAMPLES_PER_ARTIFACT:
                break
            
            # חיפוש "חכם" בתוך הזיפ - מוצא את הקובץ גם אם הוא בתוך תיקייה או בסיומת שונה
            found_path = None
            for name in all_names:
                # בודק אם ה-ID מופיע בשם הקובץ (למשל 9052650)
                if img_id in name and (name.lower().endswith('.jpg') or name.lower().endswith('.png')):
                    found_path = name
                    break
            
            if found_path:
                try:
                    with archive.open(found_path) as file:
                        img = Image.open(io.BytesIO(file.read())).convert("RGB")
                        width, height = img.size
                        
                        # סינון גודל
                        if width >= MIN_ORIGINAL_SIZE and height >= MIN_ORIGINAL_SIZE and \
                           width <= MAX_ORIGINAL_SIZE and height <= MAX_ORIGINAL_SIZE:
                            
                            img_resized = img.resize(TARGET_SIZE, Image.Resampling.LANCZOS)
                            save_name = f"{artifact_type}_{img_id}.png"
                            img_resized.save(os.path.join(output_folder, save_name))
                            extracted_count += 1
                except Exception:
                    continue
                    
        print(f"סיימתי {artifact_type}: עובדו {extracted_count} תמונות.")

print(f"\nהסתיים! בדקי אם עכשיו יש תמונות בתיקייה: {output_folder}")

סורק את קובץ הזיפ: C:\Users\linoy\OneDrive\שולחן העבודה\Samsung\crops.zip...
סיימתי Smears: עובדו 100 תמונות.
סיימתי Saturated False Color: עובדו 100 תמונות.
סיימתי Broken Line: עובדו 100 תמונות.
סיימתי Over Desaturation: עובדו 100 תמונות.
סיימתי Edge False Color: עובדו 100 תמונות.
סיימתי Corner: עובדו 100 תמונות.
סיימתי Zagging: עובדו 100 תמונות.
סיימתי Moire: עובדו 100 תמונות.
סיימתי Line Noise: עובדו 100 תמונות.
סיימתי Pattern artifact: עובדו 100 תמונות.

הסתיים! בדקי אם עכשיו יש תמונות בתיקייה: distorted_30x30_dataset
